# Cifar-100 classification via CNN.
in this notebook, we will build and train a CNN model on Cifar100 dataset


  

## phase one:
### load libraries

In [ ]:
from google.colab import drive

drive.mount("/content/drive")
%cd /content/drive/MyDrive/

In [ ]:
# Load librarys and the TensorBoard notebook extension

import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

%load_ext tensorboard


# clean log dir
!rm -rf logs/

## phase two:
### load data

In [ ]:
# load data and assert the correct number for data
## load Cifar10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
# make sure about shape of data

assert x_train.shape == (50000, 32, 32, 3)
assert y_train.shape == (50000, 1)

assert x_test.shape == (10000, 32, 32, 3)
assert y_test.shape == (10000, 1)

# create val and test set
x_val = x_test[:5000]
x_test = x_test[5000:]
y_val = y_test[:5000]
y_test = y_test[5000:]

# make sure about shape of val and test

assert x_val.shape == (5000, 32, 32, 3)
assert y_val.shape == (5000, 1)

assert x_test.shape == (5000, 32, 32, 3)
assert y_test.shape == (5000, 1)

In [ ]:
## Visualize some sample

f, axarr = plt.subplots(2, 2)
axarr[0, 0].imshow(x_train[5])
axarr[0, 0].set_title(f"label is {y_train[5]}")
axarr[0, 1].imshow(x_train[50])
axarr[0, 1].set_title(f"label is {y_train[50]}")

axarr[1, 0].imshow(x_train[500])
axarr[1, 0].set_title(f"label is {y_train[500]}")

axarr[1, 1].imshow(x_train[5000])
axarr[1, 1].set_title(f"label is {y_train[5000]}")

## phase three:
### create model
 1.  we will use panda method.
 2. would use augmention
 3. would use CNN for featur extraction
 4. would use Dense for classification

In [ ]:
# prepare layers

from tensorflow.keras import regularizers

# Define data augmentation pipeline
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.15),  # Increased rotation range
        keras.layers.RandomZoom(0.2),  # More aggressive zoom
        keras.layers.RandomContrast(0.1),  # New: Helps with lighting variations
    ]
)

# CNN + Classi fire
model = keras.Sequential(
    [
        keras.layers.InputLayer((32, 32, 3)),
        data_augmentation,
        keras.layers.Rescaling(1.0 / 255),
        # Block 1 (32 filters)
        keras.layers.Conv2D(32, (3, 3), padding="same", activation="relu"),
        keras.layers.BatchNormalization(),  # Stabilizes training
        keras.layers.Conv2D(32, (3, 3), activation="relu"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D((2, 2)),
        keras.layers.Dropout(0.25),
        # Block 2 (64 filters)
        keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(64, (3, 3), activation="relu"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D((2, 2)),
        keras.layers.Dropout(0.3),
        # Block 3 (128 filters)
        keras.layers.Conv2D(128, (3, 3), padding="same", activation="relu"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(128, (3, 3), activation="relu"),
        keras.layers.BatchNormalization(),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.4),
        # Classifier
        keras.layers.Dense(
            512, activation="relu", kernel_regularizer=regularizers.l2(0.001)
        ),
        keras.layers.Dense(
            256, activation="relu", kernel_regularizer=regularizers.l2(0.001)
        ),
        # keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(100, activation="softmax"),
    ]
)

model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Compile  model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss="sparse_categorical_crossentropy",
    metrics=["acc"],
)

In [ ]:
# custom call  back for +80 percent acc
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        """
        Halts the training when the accuravy rise  above 90%

        Args:
            epoch (integer) - index of epoch (required but unused in the function definition below)
            logs (dict) - metric results from the training epoch
        """

        # Check the loss
        if logs["val_acc"] > 80:

            # Stop if threshold is met
            print("\nLoss is lower than 0.4 so cancelling training!")
            self.model.stop_training = True


tensorboard_callback = keras.callbacks.TensorBoard(log_dir="./logs")

In [ ]:
epochs = 50

# Train the model
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    validation_data=(x_val, y_val),
    callbacks=[
        myCallback(),
        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5),
        tensorboard_callback,
    ],
    epochs=epochs,
)

In [ ]:
# Evaluate model performance on the test set
model.evaluate(x=x_test, y=y_test)

In [ ]:
# export model
model.save("Cifar100-pure.keras")

In [ ]:
%tensorboard --bind_all --logdir logs  --port 7776